In [3]:
import os
import h5py
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
import fixmat as ft
from scipy.ndimage.filters import gaussian_filter

In [9]:
#filename = '/Users/Lauren/cahv/data/etdb.hdf5'
#f = h5py.File(filename, 'r')
#baseline,meta = ft.load('/Users/Lauren/cahv/data/etdb.hdf5', 'Baseline')
#appc,meta_appc = ft.load('/Users/Lauren/cahv/data/APP_raw.hdf', "APP")
eeg,meta_eeg = ft.load('/Users/Lauren/eye-movements/data/hdf/EEG_raw.hdf', "EEG")

### Preprocessing for Folder 14

Visualizing fixation locations by drawing circles on the image

In [ ]:
img_idx = range(1, len(os.listdir("/Users/Lauren/eye-movements/data/14/"))+1)

for i in img_idx:
    test_img_data = eeg.groupby("filenumber").get_group(i)
    img = cv2.imread("/Users/Lauren/eye-movements/data/14/" + str(i) + ".jpg")
    for idx,row in test_img_data.iterrows():
        print(int(row["x"]))
        print(int(row["y"]))
        cv2.circle(img, (int(row["x"]),int(row["y"])),7,color=(0,255,0))
        
    cv2.imwrite("/Users/Lauren/eye-movements/data/viz/14/" + str(i) + ".jpg", img)

Creating the ground-truth mapping for fixation locations using a Gaussian filter:

1) add values to pixel of fixation location and surrounding pixels to account for angle of error in fixation tracking

2) Gaussian filter applied to fixation locations

In [64]:
img_idx = range(1, len(os.listdir("/Users/Lauren/eye-movements/data/14/"))+1)

for i in img_idx:

    
    img = cv2.imread("/Users/Lauren/eye-movements/data/14/" + str(i) + ".jpg")
    img_filter = np.zeros((img.shape[0],img.shape[1]))
    test_img_data = eeg.groupby("filenumber").get_group(i)
    test_img_data = test_img_data[test_img_data.y < img_filter.shape[0]]
    test_img_data = test_img_data[test_img_data.x < img_filter.shape[1]]
    test_img_data = test_img_data[test_img_data.x >= 0]
    test_img_data = test_img_data[test_img_data.y >= 0]
    
    m = range(img.shape[0])
    n = range(img.shape[1])
    for idx,row in test_img_data.iterrows():
        
        # add values to surrounding pixels according to gaussian
        y = int(row["x"])
        x = int(row["y"])
        img_filter[x,y] += 4
        top = x - 1
        bottom = x + 1
        left = y - 1
        right = y + 1
        if top in m:
            img_filter[top,y] += 2
            if left in n:
                img_filter[top,left] += 1
            if right in n:
                img_filter[top,right] += 1
        if bottom in m:
            img_filter[bottom,y] += 2
            if left in n:
                img_filter[bottom,left] += 1
            if right in n:
                img_filter[bottom,right] += 1
        if left in n:
            img_filter[x,left] += 2
        if right in n:
            img_filter[x,right] += 2
        
        
    blurred = gaussian_filter(img_filter, sigma=5)
    
    # normalize
    blurred *= (255/np.amax(blurred))
    cv2.imwrite("/Users/Lauren/eye-movements/data/filters/14/" + str(i) + ".jpg", blurred)

AttributeError: 'NoneType' object has no attribute 'shape'